In [10]:
import json
import pandas as pd

with open('../Data/BDL/gus_bdl.json', encoding='utf-8') as f:
    data = json.load(f)

In [11]:
pd.set_option("display.max_colwidth", None)
pd.DataFrame(
    [
        {
            "id": id,
            "name": s["name"],
            "kind": s["kind"],
            "dim_types": s["dimensionTypes"],
            "dims": list(map(str.splitlines, s["dimensions"].keys())),
        }
        for id, s in data["subjects"].items()
    ]
)


,id,name,kind,dim_types,dims
0,P1719,Zasoby mieszkaniowe wg form własności,Yearly,"[Formy własności, Zasoby mieszkaniowe]","[[zasoby Skarbu Państwa, mieszkania], [zasoby Skarbu Państwa, powierzchnia użytkowa mieszkań], [zasoby gminne (komunalne), izby], [zasoby gminne (komunalne), mieszkania], [zasoby gminne (komunalne), powierzchnia użytkowa mieszkań], [zasoby innych podmiotów, izby], [zasoby innych podmiotów, mieszkania], [zasoby innych podmiotów, powierzchnia użytkowa mieszkań], [zasoby osób fizycznych, izby], [zasoby osób fizycznych, mieszkania], [zasoby osób fizycznych, powierzchnia użytkowa mieszkań], [zasoby osób fizycznych w budynkach objętych wspólnotami mieszkaniowymi, mieszkania], [zasoby osób fizycznych w budynkach objętych wspólnotami mieszkaniowymi, powierzchnia użytkowa mieszkań], [zasoby spółdzielni mieszkaniowych, izby], [zasoby spółdzielni mieszkaniowych, mieszkania], [zasoby spółdzielni mieszkaniowych, powierzchnia użytkowa mieszkań], [zasoby towarzystw budownictwa społecznego (TBS), izby], [zasoby towarzystw budownictwa społecznego (TBS), mieszkania], [zasoby towarzystw budownictwa społecznego (TBS), powierzchnia użytkowa mieszkań], [zasoby w budynkach objętych wspólnotami mieszkaniowymi, mieszkania], [zasoby w budynkach objętych wspólnotami mieszkaniowymi, powierzchnia użytkowa mieszkań], [zasoby zakładów pracy, izby], [zasoby zakładów pracy, mieszkania], [zasoby zakładów pracy, powierzchnia użytkowa mieszkań]]"
1,P1727,Zasoby mieszkaniowe gminne sprzedane,Yearly,"[Rodzaje budynków, Mieszkania]","[[ogółem, mieszkania], [ogółem, powierzchnia użytkowa mieszkań], [w domach wielorodzinnych, mieszkania], [w domach wielorodzinnych, powierzchnia użytkowa mieszkań]]"
2,P1737,Mieszkania wg form własności i wyposażenia,Yearly,"[Formy własności, Wyposażenie]","[[Towarzystw Budownictwa Społecznego (TBS), centralne ogrzewanie], [Towarzystw Budownictwa Społecznego (TBS), gaz sieciowy], [Towarzystw Budownictwa Społecznego (TBS), ogółem], [Towarzystw Budownictwa Społecznego (TBS), ustęp spłukiwany], [Towarzystw Budownictwa Społecznego (TBS), ustęp spłukiwany z odprowadzeniem do sieci], [Towarzystw Budownictwa Społecznego (TBS), wodociąg], [Towarzystw Budownictwa Społecznego (TBS), wodociąg z sieci], [Towarzystw Budownictwa Społecznego (TBS), łazienka], [gmin, centralne ogrzewanie], [gmin, gaz sieciowy], [gmin, ogółem], [gmin, ustęp spłukiwany], [gmin, ustęp spłukiwany z odprowadzeniem do sieci], [gmin, wodociąg], [gmin, wodociąg z sieci], [gmin, łazienka], [innego podmiotu, centralne ogrzewanie], [innego podmiotu, gaz sieciowy], [innego podmiotu, ogółem], [innego podmiotu, ustęp spłukiwany], [innego podmiotu, ustęp spłukiwany z odprowadzeniem do sieci], [innego podmiotu, wodociąg], [innego podmiotu, wodociąg z sieci], [innego podmiotu, łazienka], [ogółem, centralne ogrzewanie], [ogółem, gaz sieciowy], [ogółem, ogółem], [ogółem, ustęp spłukiwany], [ogółem, ustęp spłukiwany z odprowadzeniem do sieci], [ogółem, wodociąg], [ogółem, wodociąg z sieci], [ogółem, łazienka], [osoby fizycznej, centralne ogrzewanie], [osoby fizycznej, gaz sieciowy], [osoby fizycznej, ogółem], [osoby fizycznej, ustęp spłukiwany], [osoby fizycznej, ustęp spłukiwany z odprowadzeniem do sieci], [osoby fizycznej, wodociąg], [osoby fizycznej, wodociąg z sieci], [osoby fizycznej, łazienka], [spółdzielni mieszkaniowej, centralne ogrzewanie], [spółdzielni mieszkaniowej, gaz sieciowy], [spółdzielni mieszkaniowej, ogółem], [spółdzielni mieszkaniowej, ustęp spłukiwany], [spółdzielni mieszkaniowej, ustęp spłukiwany z odprowadzeniem do sieci], [spółdzielni mieszkaniowej, wodociąg], [spółdzielni mieszkaniowej, wodociąg z sieci], [spółdzielni mieszkaniowej, łazienka], [zakładu pracy, centralne ogrzewanie], [zakładu pracy, gaz sieciowy], [zakładu pracy, ogółem], [zakładu pracy, ustęp spłukiwany], [zakładu pracy, ustęp spłukiwany z odprowadzeniem do sieci], [zakładu pracy, wodociąg], [zakładu pracy, wodociąg z sieci], [zakładu pracy, łazienka]]"
3,P2166,Zasoby mieszk

In [12]:
from unidecode import unidecode
import json

def voivodeship_to_enum(name):
    return unidecode(name).lower().replace("-", "")


def records_to_chartjs_datasets(sub, dim):
    datasets = {}
    for unit_id, entries in sub["dimensions"][dim]["records"].items():
        voi = voivodeship_to_enum(data["territorialUnitNames"][unit_id])
        datasets[voi] = {"label": voi}
        match sub["kind"]:
            case "Quarterly" | "Monthly":
                # Sprawdzanie czy nie zostały przypadkiem jakieś nierozdzielone zakresy w stylu "1 kwartał - 3 kwartał", ale to nie powinno się zdarzyć po wstępnym przetworzeniu danych
                for e in entries:
                    assert (
                        e["dateRange"]["contents"][0] == e["dateRange"]["contents"][1]
                    )

                datasets[voi]["data"] = [
                    {"x": e["dateRange"]["contents"][0], "y": e["value"]}
                    for e in entries
                ]
            case "Yearly":
                datasets[voi]["data"] = [
                    {"x": e["dateRange"]["contents"], "y": e["value"]} for e in entries
                ]

    return list(datasets.values())


def export_datasets(datasets, const_name):
    js = json.dumps(datasets)
    with open(
        f"../frontend/src/assets/charts/data/bdl/generated/{const_name}.ts", "w"
    ) as f:
        f.write(f"export const {const_name} = {js};\n")


In [13]:
export_datasets(records_to_chartjs_datasets(data['subjects']['P3793'], "ogółem\nogółem"), "medianaCenZaM2Datasets")